In [25]:
import seg_metrics.seg_metrics as sg
import pandas as pd
import numpy as np
import nibabel as nib
import os
import pydicom
from skimage import morphology
import matplotlib.pyplot as plt
from skimage.measure import label, regionprops
import cv2
from skimage.morphology import skeletonize, skeletonize_3d
import pickle
from sklearn.metrics import brier_score_loss
from scipy.stats import gaussian_kde
from matplotlib.patches import Rectangle
from skimage import morphology
import glob
from sklearn.metrics import r2_score
import scipy
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
import seaborn as sns
from scipy import stats
from scipy import ndimage
import numpy as np
import SimpleITK as sitk

In [49]:
import lookup_tables

In [116]:
def icc(Y, icc_type='ICC(2,1)'):
    ''' Calculate intraclass correlation coefficient

    ICC Formulas are based on:
    Shrout, P. E., & Fleiss, J. L. (1979). Intraclass correlations: uses in
    assessing rater reliability. Psychological bulletin, 86(2), 420.
    icc1:  x_ij = mu + beta_j + w_ij
    icc2/3:  x_ij = mu + alpha_i + beta_j + (ab)_ij + epsilon_ij
    Code modifed from nipype algorithms.icc
    https://github.com/nipy/nipype/blob/master/nipype/algorithms/icc.py

    Args:
        Y: The data Y are entered as a 'table' ie. subjects are in rows and repeated
            measures in columns
        icc_type: type of ICC to calculate. (ICC(2,1), ICC(2,k), ICC(3,1), ICC(3,k)) 
    Returns:
        ICC: (np.array) intraclass correlation coefficient
    '''

    [n, k] = Y.shape

    # Degrees of Freedom
    dfc = k - 1
    dfe = (n - 1) * (k-1)
    dfr = n - 1

    # Sum Square Total
    mean_Y = np.mean(Y)
    SST = ((Y - mean_Y) ** 2).sum()

    # create the design matrix for the different levels
    x = np.kron(np.eye(k), np.ones((n, 1)))  # sessions
    x0 = np.tile(np.eye(n), (k, 1))  # subjects
    X = np.hstack([x, x0])

    # Sum Square Error
    predicted_Y = np.dot(np.dot(np.dot(X, np.linalg.pinv(np.dot(X.T, X))),
                                X.T), Y.flatten('F'))
    residuals = Y.flatten('F') - predicted_Y
    SSE = (residuals ** 2).sum()

    MSE = SSE / dfe

    # Sum square column effect - between colums
    SSC = ((np.mean(Y, 0) - mean_Y) ** 2).sum() * n
    MSC = SSC / dfc  # / n (without n in SPSS results)

    # Sum Square subject effect - between rows/subjects
    SSR = SST - SSC - SSE
    MSR = SSR / dfr

    if icc_type == 'icc1':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        # ICC = (MSR - MSRW) / (MSR + (k-1) * MSRW)
        NotImplementedError("This method isn't implemented yet.")

    elif icc_type == 'ICC(2,1)' or icc_type == 'ICC(2,k)':
        # ICC(2,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error +
        # k*(mean square columns - mean square error)/n)
        if icc_type == 'ICC(2,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE + k * (MSC - MSE) / n)

    elif icc_type == 'ICC(3,1)' or icc_type == 'ICC(3,k)':
        # ICC(3,1) = (mean square subject - mean square error) /
        # (mean square subject + (k-1)*mean square error)
        if icc_type == 'ICC(3,k)':
            k = 1
        ICC = (MSR - MSE) / (MSR + (k-1) * MSE)

    return ICC

def cl_score(v, s):
    """[this function computes the skeleton volume overlap]
    Args:
        v ([bool]): [image]
        s ([bool]): [skeleton]
    Returns:
        [float]: [computed skeleton volume intersection]
    """
    return np.sum(v*s)/np.sum(s)


def clDice(v_l, v_p):
    """[this function computes the cldice metric]
    Args:
        v_p ([bool]): [predicted image]
        v_l ([bool]): [ground truth image]
    Returns:
        [float]: [cldice metric]
    """
    if len(v_p.shape)==2:
        tprec = cl_score(v_p,skeletonize(v_l))
        tsens = cl_score(v_l,skeletonize(v_p))
    elif len(v_p.shape)==3:
        tprec = cl_score(v_p,skeletonize_3d(v_l))
        tsens = cl_score(v_l,skeletonize_3d(v_p))
    return 2*tprec*tsens/(tprec+tsens)

In [66]:
def compute_dice_coef(y_true, y_pred):

    smooth=1
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [121]:
def compute_jaccard_coef(y_true, y_pred):

    smooth=1
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection
    return intersection / union

In [100]:
def getEdgeOfMask(mask):
    
    edge = np.zeros_like(mask)
    mask_pixels = np.where(mask > 0)

    for idx in range(0,mask_pixels[0].size):

        x = mask_pixels[0][idx]
        y = mask_pixels[1][idx]
        z = mask_pixels[2][idx]

        if mask[x-1:x+2, y-1:y+2, z-1:z+2].sum() < 27:
            edge[x,y,z] = 1
            
    return edge

def compute_AddedPathLength(mask_true, mask_pred, spacing_mm):
    
    edge_true = getEdgeOfMask(mask_true)
    edge_pred = getEdgeOfMask(mask_pred)
   
    apl = (edge_true > edge_pred).astype(int).sum()
    
    return apl*spacing_mm[0]*spacing_mm[1]*spacing_mm[2]/10 

In [105]:
def compute_nsd(img_1, img_2, tau):
    
    img_1_b = getEdgeOfMask(img_1)
    img_2_b = getEdgeOfMask(img_2)
    
    strel_size = 1 + tau*2
    strel = np.ones((strel_size, strel_size, strel_size))
    
    img_1_bb = morphology.binary_dilation(img_1_b, strel)
    img_2_bb = morphology.binary_dilation(img_2_b, strel)
    
    int_1 = img_1_b*img_2_bb
    int_2 = img_1_bb*img_2_b
    
    return (np.sum(int_1)+np.sum(int_2))/(np.sum(img_1_b) + np.sum(img_2_b))

In [106]:
def compute_hausdorf_surf_distance(mask_gt, mask_pred, spacing_mm, percent=95):
    """Computes the robust Hausdorff distance. "Robust", because it uses the `percent` percentile 
    of the distances instead of the maximum distance. The percentage is computed by correctly taking 
    the area of each surface element into account.
    Args:
    mask_gt: 3-dim Numpy array of type bool. The ground truth mask.
    mask_pred: 3-dim Numpy array of type bool. The predicted mask.
    spacing_mm: 3-element list-like structure. Voxel spacing in x0, x1 and x2 direction.
    percent: a float value between 0 and 100.
    Returns:
    a float value. The robust Hausdorff distance in mm. If one of the masks
    is empty, the corresponding lists are empty and all distances in the other
    list are `inf`.
    """

    neighbour_code_to_surface_area = np.zeros([256])
    for code in range(256):
        normals = np.array(lookup_tables._NEIGHBOUR_CODE_TO_NORMALS[code])
        sum_area = 0
        for normal_idx in range(normals.shape[0]):
            # normal vector
            n = np.zeros([3])
            n[0] = normals[normal_idx, 0] * spacing_mm[1] * spacing_mm[2]
            n[1] = normals[normal_idx, 1] * spacing_mm[0] * spacing_mm[2]
            n[2] = normals[normal_idx, 2] * spacing_mm[0] * spacing_mm[1]
            area = np.linalg.norm(n)
            sum_area += area
        neighbour_code_to_surface_area[code] = sum_area

    # compute the bounding box of the masks to trim
    # the volume to the smallest possible processing subvolume
    mask_all = mask_gt | mask_pred
    bbox_min = np.zeros(3, np.int64)
    bbox_max = np.zeros(3, np.int64)

    # max projection to the x0-axis
    proj_0 = np.max(np.max(mask_all, axis=2), axis=1)
    idx_nonzero_0 = np.nonzero(proj_0)[0]
    if len(idx_nonzero_0) == 0:  # pylint: disable=g-explicit-length-test
        return {"distances_gt_to_pred": np.array([]),
                "distances_pred_to_gt": np.array([]),
                "surfel_areas_gt": np.array([]),
                "surfel_areas_pred": np.array([])}

    bbox_min[0] = np.min(idx_nonzero_0)
    bbox_max[0] = np.max(idx_nonzero_0)

    # max projection to the x1-axis
    proj_1 = np.max(np.max(mask_all, axis=2), axis=0)
    idx_nonzero_1 = np.nonzero(proj_1)[0]
    bbox_min[1] = np.min(idx_nonzero_1)
    bbox_max[1] = np.max(idx_nonzero_1)

    # max projection to the x2-axis
    proj_2 = np.max(np.max(mask_all, axis=1), axis=0)
    idx_nonzero_2 = np.nonzero(proj_2)[0]
    bbox_min[2] = np.min(idx_nonzero_2)
    bbox_max[2] = np.max(idx_nonzero_2)

    # crop the processing subvolume.
    # we need to zeropad the cropped region with 1 voxel at the lower,
    # the right and the back side. This is required to obtain the "full"
    # convolution result with the 2x2x2 kernel
    cropmask_gt = np.zeros((bbox_max - bbox_min)+2, np.uint8)
    cropmask_pred = np.zeros((bbox_max - bbox_min)+2, np.uint8)

    cropmask_gt[0:-1, 0:-1, 0:-1] = mask_gt[bbox_min[0]:bbox_max[0]+1,bbox_min[1]:bbox_max[1]+1,
                                          bbox_min[2]:bbox_max[2]+1]

    cropmask_pred[0:-1, 0:-1, 0:-1] = mask_pred[bbox_min[0]:bbox_max[0]+1,bbox_min[1]:bbox_max[1]+1,
                                              bbox_min[2]:bbox_max[2]+1]

    # compute the neighbour code (local binary pattern) for each voxel
    # the resultsing arrays are spacially shifted by minus half a voxel in each
    # axis.
    # i.e. the points are located at the corners of the original voxels
    kernel = np.array([[[128, 64],[32, 16]],[[8, 4],[2, 1]]])
    
    neighbour_code_map_gt = ndimage.filters.correlate(cropmask_gt.astype(np.uint8), kernel, mode="constant", cval=0)
    neighbour_code_map_pred = ndimage.filters.correlate(cropmask_pred.astype(np.uint8), kernel, mode="constant", cval=0)

    # create masks with the surface voxels
    borders_gt = ((neighbour_code_map_gt != 0) & (neighbour_code_map_gt != 255))
    borders_pred = ((neighbour_code_map_pred != 0) &(neighbour_code_map_pred != 255))

    # compute the distance transform (closest distance of each voxel to the surface voxels)
    if borders_gt.any():
        distmap_gt = ndimage.morphology.distance_transform_edt(~borders_gt, sampling=spacing_mm)
    else:
        distmap_gt = np.Inf * np.ones(borders_gt.shape)

    if borders_pred.any():
        distmap_pred = ndimage.morphology.distance_transform_edt(~borders_pred, sampling=spacing_mm)
    else:
        distmap_pred = np.Inf * np.ones(borders_pred.shape)

    # compute the area of each surface element
    surface_area_map_gt = neighbour_code_to_surface_area[neighbour_code_map_gt]
    surface_area_map_pred = neighbour_code_to_surface_area[neighbour_code_map_pred]

    # create a list of all surface elements with distance and area
    distances_gt_to_pred = distmap_pred[borders_gt]
    distances_pred_to_gt = distmap_gt[borders_pred]
    surfel_areas_gt = surface_area_map_gt[borders_gt]
    surfel_areas_pred = surface_area_map_pred[borders_pred]

    # sort them by distance
    if distances_gt_to_pred.shape != (0,):
        sorted_surfels_gt = np.array(sorted(zip(distances_gt_to_pred, surfel_areas_gt)))
        distances_gt_to_pred = sorted_surfels_gt[:, 0]
        surfel_areas_gt = sorted_surfels_gt[:, 1]

    if distances_pred_to_gt.shape != (0,):
        sorted_surfels_pred = np.array(sorted(zip(distances_pred_to_gt, surfel_areas_pred)))
        distances_pred_to_gt = sorted_surfels_pred[:, 0]
        surfel_areas_pred = sorted_surfels_pred[:, 1]

    if len(distances_gt_to_pred) > 0:  # pylint: disable=g-explicit-length-test
        surfel_areas_cum_gt = np.cumsum(surfel_areas_gt) / np.sum(surfel_areas_gt)
        idx = np.searchsorted(surfel_areas_cum_gt, percent/100.0)
        perc_distance_gt_to_pred = distances_gt_to_pred[min(idx, len(distances_gt_to_pred)-1)]
        max_distance_gt_to_pred = np.max(distances_gt_to_pred)
    else:
        perc_distance_gt_to_pred = np.Inf
        max_distance_gt_to_pred = np.Inf

    if len(distances_pred_to_gt) > 0:  # pylint: disable=g-explicit-length-test
        surfel_areas_cum_pred = (np.cumsum(surfel_areas_pred) /np.sum(surfel_areas_pred))
        idx = np.searchsorted(surfel_areas_cum_pred, percent/100.0)
        perc_distance_pred_to_gt = distances_pred_to_gt[min(idx, len(distances_pred_to_gt)-1)]
        max_distance_pred_to_gt = np.max(distances_pred_to_gt)
    else:
        perc_distance_pred_to_gt = np.Inf
        max_distance_pred_to_gt = np.Inf

    return max(max_distance_gt_to_pred, max_distance_pred_to_gt), max(perc_distance_gt_to_pred, perc_distance_pred_to_gt)

In [126]:
sub_names_test = ['AMC012', 'AMC006', 
                  'MUMC094', 'MUMC027', 'MUMC079', 'MUMC052', 'MUMC127', 'MUMC071', 'MUMC038', 'MUMC093', 'MUMC107', 
                  'MUMC022', 'MUMC114', 'MUMC115', 'MUMC069', 'MUMC130', 'MUMC036', 'MUMC007', 'MUMC059', 'MUMC080', 
                  'UMCU036', 'UMCU025', 'UMCU008', 'UMCU034']

sub_names_emc = ['EMC003', 'EMC004', 'EMC005', 'EMC007', 'EMC008', 'EMC009', 'EMC011', 
                 'EMC015', 'EMC018', 'EMC020', 'EMC024', 'EMC027', 'EMC029', 'EMC031', 
                 'EMC032', 'EMC034', 'EMC035', 'EMC036', 'EMC038', 'EMC041', 'EMC042', 
                 'EMC043', 'EMC045', 'EMC046', 'EMC047', 'EMC048', 'EMC049', 'EMC050', 
                 'EMC051', 'EMC052', 'EMC054', 'EMC055', 'EMC056', 'EMC057']

In [133]:
nifti_dirname_GT_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_GT"
nifti_dirname_GT_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_GT"
nifti_dirname_GT_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_GT"
nifti_dirname_GT_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_GT"

nifti_dirname_nnunet_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_nnunet"
nifti_dirname_nnunet_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_nnunet"
nifti_dirname_nnunet_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_nnunet"
nifti_dirname_nnunet_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_nnunet"

nifti_dirname_nnunet_test_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_nnunet_p"
nifti_dirname_nnunet_t2w_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_nnunet_p"
nifti_dirname_nnunet_t1wce_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_nnunet_p"
nifti_dirname_nnunet_emc_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_nnunet_p"

nifti_dirname_plaqunet_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_plaqunet"
nifti_dirname_plaqunet_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_plaqunet"
nifti_dirname_plaqunet_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_plaqunet"
nifti_dirname_plaqunet_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_plaqunet"

nifti_dirname_plaqunet_test_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_plaqunet_p"
nifti_dirname_plaqunet_t2w_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_plaqunet_p"
nifti_dirname_plaqunet_t1wce_sm=r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_plaqunet_p"
nifti_dirname_plaqunet_emc_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_plaqunet_p"

nifti_dirname_plaqumap_test = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_plaqumap"
nifti_dirname_plaqumap_t2w = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_plaqumap"
nifti_dirname_plaqumap_t1wce = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_plaqumap"
nifti_dirname_plaqumap_emc = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_plaqumap"

nifti_dirname_plaqumap_test_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\test\test_plaqumap_p"
nifti_dirname_plaqumap_t2w_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t2w\t2w_plaqumap_p"
nifti_dirname_plaqumap_t1wce_sm=r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\t1wce\t1wce_plaqumap_p"
nifti_dirname_plaqumap_emc_sm = r"C:\Users\E.Lavrova\Documents\GitHub\plaqueuqalp\res\nifti_compare\emc\emc_plaqumap_p"

In [136]:
def get_scores_df(sub_names, dirname_gt, dirname_pred):

    df_scores = []

    for sub_name in sub_names:
        
        print (sub_name)

        filename_gt = os.path.join(dirname_gt, sub_name + '.nii.gz')
        filename_pred = os.path.join(dirname_pred, sub_name + '.nii.gz')

        mask_gt = sitk.ReadImage(filename_gt)
        mask_pred = sitk.ReadImage(filename_pred)

        mask_gt = sitk.GetArrayFromImage(mask_gt)
        mask_pred = sitk.GetArrayFromImage(mask_pred)

        rec = {'sub': sub_name} 
        rec['DSC'] = compute_dice_coef(mask_gt, mask_pred)
        rec['JSc'] = compute_jaccard_coef(mask_gt, mask_pred)

        hd_max, hd_95 = compute_hausdorf_surf_distance(mask_gt, mask_pred, [2, 0.303030, 0.303030], percent=95)

        rec['hd'] = hd_max
        rec['hd95'] = hd_95

        rec['clDice'] = clDice(mask_gt, mask_pred)
        rec['nsd'] = compute_nsd(mask_gt, mask_pred, tau=1)
        rec['apl'] = compute_AddedPathLength(mask_gt, mask_pred, [2, 0.303030, 0.303030])

        rec['vol_gt'] = np.sum(mask_gt)
        rec['vol_pred'] = np.sum(mask_pred)

        rec['vol_diff'] = rec['vol_gt']-rec['vol_pred']
        rec['abs_vol_diff'] = abs(rec['vol_gt']-rec['vol_pred'])

        df_scores.append(rec)

    df_scores = pd.DataFrame(df_scores_test_1)

    return df_scores

In [ ]:
df_scores_nnunet_test = get_scores_df(sub_names_test, nifti_dirname_GT_test, nifti_dirname_nnunet_test)
df_scores_nnunet_t1wce = get_scores_df(sub_names_test, nifti_dirname_GT_t1wce, nifti_dirname_nnunet_t1wce)
df_scores_nnunet_t2w = get_scores_df(sub_names_test, nifti_dirname_GT_t2w, nifti_dirname_nnunet_t2w)
df_scores_nnunet_emc = get_scores_df(sub_names_emc, nifti_dirname_GT_emc, nifti_dirname_nnunet_emc)

AMC012
AMC006
MUMC094
MUMC027


C:\ProgramData\Anaconda3\envs\segway\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in ulong_scalars
C:\ProgramData\Anaconda3\envs\segway\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: overflow encountered in ulong_scalars


MUMC079
MUMC052
MUMC127
MUMC071
MUMC038
MUMC093
MUMC107
MUMC022
MUMC114
MUMC115
MUMC069
MUMC130
MUMC036
MUMC007
MUMC059
MUMC080
UMCU036
UMCU025
UMCU008
UMCU034
